In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import torch.nn.functional as F
from focal import FocalLoss
print(torch.__version__)
import torch
torch.manual_seed(0)

1.8.1+cu111


In [2]:
training = os.listdir('../train/')
study_label = pd.read_csv('../archive/train_study_level.csv')
image_label = pd.read_csv('../archive/train_image_level.csv')
paths = []
labels = []
dct = pd.read_csv('train.csv', index_col=0)
dct['image'] = dct.image.apply(lambda x: x[:-4])
dct = dct.set_index('study').to_dict()['image']

In [3]:
for index, row in study_label.iterrows():
    name = dct[row['id'].replace('_study', '')] + '.png'
    if name in training:
        paths.append('../train/' + name)
        if row['Negative for Pneumonia'] == 1:
            labels.append(0)
        elif row['Typical Appearance'] == 1:
            labels.append(1)
        elif row['Indeterminate Appearance'] == 1:
            labels.append(2)
        elif row['Atypical Appearance'] == 1:
            labels.append(3)
    else:
        print(name)

In [4]:
class Net(nn.Module):
    def __init__(self, out_size, model):
        super(Net, self).__init__()
        if model == 'dense':
            self.model = torchvision.models.densenet121(pretrained=True, **{'drop_rate' : 0.3})
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(
                nn.Linear(num_ftrs, out_size),
                nn.Sigmoid()
            )
            
        elif model == 'res':
            self.model = torchvision.models.wide_resnet101_2(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Sequential(
                nn.Linear(num_ftrs, out_size),
                nn.Sigmoid()
            )
        elif model == 'inception':
            self.model = torchvision.models.inception_v3(pretrained=True, **{"aux_logits": False})
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Sequential(
                nn.Linear(num_ftrs, out_size)
            )
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
class classification(nn.Module):
    def __init__(self, paths, labels, size=(512,512)):
        self.paths = paths
        self.labels = labels
        self.example = []
        self.size =size
    
    def __getitem__(self, idx):
            path = self.paths[idx]
            img = cv2.imread(path)
            R, G, B = cv2.split(img)
            output1_R = cv2.equalizeHist(R)
            output1_G = cv2.equalizeHist(G)
            output1_B = cv2.equalizeHist(B)

            img = cv2.merge((output1_R, output1_G, output1_B))
            img = (img - np.mean(img))/np.std(img)
            img = cv2.resize(img, self.size)
            x = torch.from_numpy(np.array(img)).view((3, self.size[0], self.size[1]))
            x = x.float()
            y = self.labels[idx]
            y = torch.tensor(y)
            return x, y
        
    def __len__(self):
        return len(self.paths)
    
    def get(self):
        return self.example

In [6]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = Net(4, 'inception')
model.load_state_dict(torch.load('./model_weights/inception_after_init_lgd_focal.pt'))
model.to(device)

Net(
  (model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): BasicConv2d(


In [7]:
dataset = classification(paths, labels, size=(512, 512))
print(len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [5200, 854])

train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                            batch_size=16,
                                            shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=val_set,
                                            batch_size=16,
                                            shuffle=False)

6054


In [9]:
model.eval()    

test_output = []
test_true = []
train_output = []
train_true = []

for i, (data,labels) in tqdm(enumerate(test_loader),
                                       total = len(test_loader)):
    data, labels = data.to(device),labels.to(device)

    with torch.no_grad():
        outputs = model(data)

    outputs = torch.nn.functional.softmax(outputs, 1)
    _, predicted = torch.max(outputs.data, 1)
    test_output.append(predicted.detach().cpu().numpy())
    test_true.append(labels.detach().cpu().numpy())
    
for i, (data,labels) in tqdm(enumerate(train_loader),
                                       total = len(train_loader)):
    data, labels = data.to(device),labels.to(device)

    with torch.no_grad():
        outputs = model(data)

    outputs = torch.nn.functional.softmax(outputs, 1)
    _, predicted = torch.max(outputs.data, 1)
    train_output.append(predicted.detach().cpu().numpy())
    train_true.append(labels.detach().cpu().numpy())
test_output = np.concatenate(test_output)
test_true = np.concatenate(test_true)
train_output = np.concatenate(train_output)
train_true = np.concatenate(train_true)

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_output, test_true, normalize='true')

array([[0.64583333, 0.17083333, 0.11666667, 0.06666667],
       [0.12736661, 0.62994836, 0.16179002, 0.08089501],
       [0.15151515, 0.09090909, 0.45454545, 0.3030303 ],
       [0.        , 0.        , 0.        , 0.        ]])

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(train_output, train_true, normalize='true')

array([[0.60725859, 0.17303953, 0.15878159, 0.06092029],
       [0.14199656, 0.61703959, 0.16666667, 0.07429719],
       [0.05988024, 0.16167665, 0.51497006, 0.26347305],
       [0.        , 0.        , 0.        , 1.        ]])